In [1]:
# Imports and Setup
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
)

RANDOM_STATE = 42  # For reproducibility

In [2]:
# Load and Clean Dataset
# Expected columns: "Category" (ham/spam), "Message" (text)
df = pd.read_csv("mail_l7_dataset.csv")

# Basic cleaning: replace NaNs with empty strings.
df = df.where(pd.notnull(df), "")

# Encode labels: spam -> 0, ham -> 1 
df.loc[df["Category"].str.lower().str.strip() == "spam", "Category"] = 0
df.loc[df["Category"].str.lower().str.strip() == "ham",  "Category"] = 1

df.head()

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
# Split features (X) and target (y)
X = df["Message"].astype(str)
y = df["Category"].astype(int)

In [4]:
# Train/test split 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print("=== SPLIT SIZES ===")
print("Train:", X_train.shape[0], " | Test:", X_test.shape[0])

=== SPLIT SIZES ===
Train: 4457  | Test: 1115


In [5]:

tfidf = TfidfVectorizer(min_df=1, stop_words="english", lowercase=True)
X_train_features = tfidf.fit_transform(X_train)
X_test_features  = tfidf.transform(X_test)

print("\n=== TF-IDF SHAPES ===")
print("X_train:", X_train_features.shape, " | X_test:", X_test_features.shape)


=== TF-IDF SHAPES ===
X_train: (4457, 7440)  | X_test: (1115, 7440)


In [6]:
# Train Logistic Regression (baseline)
lr = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
lr.fit(X_train_features, y_train)
lr_pred = lr.predict(X_test_features)


In [7]:

rf = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE)
rf.fit(X_train_features, y_train)
rf_pred = rf.predict(X_test_features.toarray())


In [8]:

nb = MultinomialNB()
nb.fit(X_train_features, y_train)
nb_pred = nb.predict(X_test_features)

In [9]:
#  Print main metrics
def print_metrics(model_name, y_true, y_pred):
    print(f"\n{model_name} Performance:")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, pos_label=0))
    print("Recall   :", recall_score(y_true, y_pred, pos_label=0))
    print("F1-score :", f1_score(y_true, y_pred, pos_label=0))

#  Print confusion matrix
def show_confusion(model_name, y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
    print(f"\n{model_name} Confusion Matrix:")
    print(pd.DataFrame(
        cm,
        index=["Actual: Ham (1)", "Actual: Spam (0)"],
        columns=["Pred: Ham (1)", "Pred: Spam (0)"]
    ))

In [11]:
print_metrics("Logistic Regression", y_test, lr_pred)
show_confusion("Logistic Regression", y_test, lr_pred)



Logistic Regression Performance:
Accuracy : 0.967713004484305
Precision: 1.0
Recall   : 0.7583892617449665
F1-score : 0.8625954198473282

Logistic Regression Confusion Matrix:
                  Pred: Ham (1)  Pred: Spam (0)
Actual: Ham (1)             966               0
Actual: Spam (0)             36             113


In [12]:
print_metrics("Random Forest", y_test, rf_pred)
show_confusion("Random Forest", y_test, rf_pred)



Random Forest Performance:
Accuracy : 0.9829596412556054
Precision: 1.0
Recall   : 0.87248322147651
F1-score : 0.931899641577061

Random Forest Confusion Matrix:
                  Pred: Ham (1)  Pred: Spam (0)
Actual: Ham (1)             966               0
Actual: Spam (0)             19             130


In [13]:
print_metrics("Naive Bayes", y_test, nb_pred)
show_confusion("Naive Bayes", y_test, nb_pred)


Naive Bayes Performance:
Accuracy : 0.9766816143497757
Precision: 1.0
Recall   : 0.825503355704698
F1-score : 0.9044117647058824

Naive Bayes Confusion Matrix:
                  Pred: Ham (1)  Pred: Spam (0)
Actual: Ham (1)             966               0
Actual: Spam (0)             26             123
